In [72]:
# import data manipulation tools
import numpy as np
import pandas as pd
# import data wrangling functions
import wrangle as w
# import nlp word counting functions
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# import classification models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
# import classification metrics
from sklearn.metrics import accuracy_score
# ignore warning mesages
import warnings
warnings.filterwarnings("ignore")

In [2]:
readmes = w.post_explore_wrangle_github_repositories()

In [3]:
readmes.head(3)

,repo,language,readme_contents,cleaned_readme_contents
0,huggingface/transformers,Python,<!---\nCopyright 2020 The HuggingFace Team. Al...,copyright 2020 huggingface team right reserved...
1,apachecn/ailearning,Python,"<p align=""center"">\n <a href=""https://www.a...",p aligncenter hrefhttpswwwapachecnorg img widt...
2,google-research/bert,Python,"# BERT\n\n**\*\*\*\*\* New March 11th, 2020: S...",bert new march 11th 2020 smaller bert model re...


In [4]:
# remove null rows (adding this to prep stage)
# readmes = readmes[readmes.cleaned_readme_contents.isna() == False]

## Split data

In [5]:
train, validate, test = w.train_split(readmes)

In [29]:
readmes.shape, train.shape, validate.shape, test.shape

((499, 4), (349, 4), (100, 4), (50, 4))

In [7]:
X_train = train.cleaned_readme_contents
y_train = train.language
X_validate = validate.cleaned_readme_contents
y_validate = validate.language
X_test = test.cleaned_readme_contents
y_test = test.language

## Baseline

In [8]:
train.language.value_counts(normalize=True)

Python    0.598854
HTML      0.401146
Name: language, dtype: float64

In [9]:
train.language.value_counts(normalize=True)[0]

0.5988538681948424

In [10]:
baseline = 'Python'

In [11]:
accuracy = train.language.value_counts(normalize=True)[0]
acc_validate = validate.language.value_counts(normalize=True)[0]
metric_df = pd.DataFrame(data=[
    {
        'model' : 'baseline',
        'accuracy' : accuracy,
        'acc-validate' : acc_validate,
        'difference' : acc_validate - accuracy,
    }
])

In [12]:
metric_df

,model,accuracy,acc-validate,difference
0,baseline,0.598854,0.6,0.001146


## The baseline is 'Python' with an accuacy of 60%

# Modeling

## CountVectorizer models

In [13]:
# make a Count Vectorizer object
cv = CountVectorizer()
# use the thing
X_bow = cv.fit_transform(X_train)
tree = DecisionTreeClassifier(max_depth=3)
tree.fit(X_bow, y_train)
tree.score(X_bow, y_train)

0.8681948424068768

In [14]:
# bag_of_words_cv

In [15]:
# make a Tfidf Vectorizer object
tfidf = TfidfVectorizer()
# use the thing
bag_of_words_tfidf = tfidf.fit_transform(X_train)

In [16]:
cv = CountVectorizer(ngram_range=(1, 3))
bag_of_grams = cv.fit_transform(X_train)

In [17]:
tfidf = CountVectorizer(ngram_range=(2, 3))
bag_of_grams = tfidf.fit_transform(X_train)

In [18]:
for i in range(1,4):
    for j in range(3,8):
        cv = CountVectorizer(ngram_range=(i, i))
        X_bow = cv.fit_transform(X_train)
        X_val_bow = cv.transform(X_validate)
        tree = DecisionTreeClassifier(max_depth=j)
        tree.fit(X_bow, y_train)
        accuracy = tree.score(X_bow, y_train)
        acc_validate = tree.score(X_val_bow, y_validate)
        
        metric_df = metric_df.append(
            {
                'model' : f'decistion_tree-cv_{i}gram_{j}depth',
                'accuracy' : accuracy,
                'acc-validate' : acc_validate,
                'difference' : acc_validate - accuracy,
            }, ignore_index=True
        )

In [19]:
metric_df

,model,accuracy,acc-validate,difference
0,baseline,0.598854,0.60,0.001146
1,decistion_tree-cv_1gram_3depth,0.868195,0.90,0.031805
2,decistion_tree-cv_1gram_4depth,0.882521,0.90,0.017479
3,decistion_tree-cv_1gram_5depth,0.914040,0.88,-0.034040
4,decistion_tree-cv_1gram_6depth,0.936963,0.88,-0.056963
5,decistion_tree-cv_1gram_7depth,0.954155,0.90,-0.054155
6,decistion_tree-cv_2gram_3depth,0.805158,0.74,-0.065158
7,decistion_tree-cv_2gram_4depth,0.833811,0.76,-0.073811
8,decistion_tree-cv_2gram_5depth,0.853868,0.77,-0.083868
9,decistion_tree-cv_2gram_6depth,0.868195,0.78,-0.088195


In [20]:
for i in range(1,4):
    for j in range(3,8):
        cv = CountVectorizer(ngram_range=(i, i))
        X_bow = cv.fit_transform(X_train)
        X_val_bow = cv.transform(X_validate)
        tree = RandomForestClassifier(max_depth=j)
        tree.fit(X_bow, y_train)
        accuracy = tree.score(X_bow, y_train)
        acc_validate = tree.score(X_val_bow, y_validate)
        
        metric_df = metric_df.append(
            {
                'model' : f'random_forest-cv_{i}gram_{j}depth',
                'accuracy' : accuracy,
                'acc-validate' : acc_validate,
                'difference' : acc_validate - accuracy,
            }, ignore_index=True
        )

In [21]:
for i in range(1,4):
    for j in range(3,8):
        cv = CountVectorizer(ngram_range=(i, i))
        X_bow = cv.fit_transform(X_train)
        X_val_bow = cv.transform(X_validate)
        model = KNeighborsClassifier(n_neighbors=j)
        model.fit(X_bow, y_train)
        accuracy = model.score(X_bow, y_train)
        acc_validate = model.score(X_val_bow, y_validate)
        
        metric_df = metric_df.append(
            {
                'model' : f'k_nearest-cv_{i}gram_{j}neighbors',
                'accuracy' : accuracy,
                'acc-validate' : acc_validate,
                'difference' : acc_validate - accuracy,
            }, ignore_index=True
        )

In [22]:
for i in range(1,4):
    cv = CountVectorizer(ngram_range=(i, i))
    X_bow = cv.fit_transform(X_train)
    X_val_bow = cv.transform(X_validate)
    model = LogisticRegression()
    model.fit(X_bow, y_train)
    accuracy = model.score(X_bow, y_train)
    acc_validate = model.score(X_val_bow, y_validate)

    metric_df = metric_df.append(
        {
            'model' : f'logistic_regress-cv_{i}gram',
            'accuracy' : accuracy,
            'acc-validate' : acc_validate,
            'difference' : acc_validate - accuracy,
        }, ignore_index=True
    )

# TfidfVectorizer Models

In [23]:
for i in range(1,4):
    for j in range(3,8):
        tfidf = TfidfVectorizer(ngram_range=(i, i))
        X_bow = tfidf.fit_transform(X_train)
        X_val_bow = tfidf.transform(X_validate)
        tree = DecisionTreeClassifier(max_depth=j)
        tree.fit(X_bow, y_train)
        accuracy = tree.score(X_bow, y_train)
        acc_validate = tree.score(X_val_bow, y_validate)
        
        metric_df = metric_df.append(
            {
                'model' : f'decistion_tree-tfidf_{i}gram_{j}depth',
                'accuracy' : accuracy,
                'acc-validate' : acc_validate,
                'difference' : acc_validate - accuracy,
            }, ignore_index=True
        )

In [24]:
for i in range(1,4):
    for j in range(3,8):
        tfidf = TfidfVectorizer(ngram_range=(i, i))
        X_bow = tfidf.fit_transform(X_train)
        X_val_bow = tfidf.transform(X_validate)
        tree = RandomForestClassifier(max_depth=j)
        tree.fit(X_bow, y_train)
        accuracy = tree.score(X_bow, y_train)
        acc_validate = tree.score(X_val_bow, y_validate)
        
        metric_df = metric_df.append(
            {
                'model' : f'random_forest-tfidf_{i}gram_{j}depth',
                'accuracy' : accuracy,
                'acc-validate' : acc_validate,
                'difference' : acc_validate - accuracy,
            }, ignore_index=True
        )

In [25]:
for i in range(1,4):
    for j in range(3,8):
        tfidf = TfidfVectorizer(ngram_range=(i, i))
        X_bow = tfidf.fit_transform(X_train)
        X_val_bow = tfidf.transform(X_validate)
        model = KNeighborsClassifier(n_neighbors=j)
        model.fit(X_bow, y_train)
        accuracy = model.score(X_bow, y_train)
        acc_validate = model.score(X_val_bow, y_validate)
        
        metric_df = metric_df.append(
            {
                'model' : f'k_nearest-tfidf_{i}gram_{j}neighbors',
                'accuracy' : accuracy,
                'acc-validate' : acc_validate,
                'difference' : acc_validate - accuracy,
            }, ignore_index=True
        )

In [26]:
for i in range(1,4):
    tfidf = TfidfVectorizer(ngram_range=(i, i))
    X_bow = tfidf.fit_transform(X_train)
    X_val_bow = tfidf.transform(X_validate)
    model = LogisticRegression()
    model.fit(X_bow, y_train)
    accuracy = model.score(X_bow, y_train)
    acc_validate = model.score(X_val_bow, y_validate)

    metric_df = metric_df.append(
        {
            'model' : f'logistic_regress-tfidf_{i}gram',
            'accuracy' : accuracy,
            'acc-validate' : acc_validate,
            'difference' : acc_validate - accuracy,
        }, ignore_index=True
    )

In [27]:
metric_df

,model,accuracy,acc-validate,difference
0,baseline,0.598854,0.60,0.001146
1,decistion_tree-cv_1gram_3depth,0.868195,0.90,0.031805
2,decistion_tree-cv_1gram_4depth,0.882521,0.90,0.017479
3,decistion_tree-cv_1gram_5depth,0.914040,0.88,-0.034040
4,decistion_tree-cv_1gram_6depth,0.936963,0.88,-0.056963
...,...,...,...,...
92,k_nearest-tfidf_3gram_6neighbors,0.401146,0.40,-0.001146
93,k_nearest-tfidf_3gram_7neighbors,0.401146,0.40,-0.001146
94,logistic_regress-tfidf_1gram,0.954155,0.79,-0.164155
95,logistic_regress-tfidf_2gram,0.971347,0.60,-0.371347


In [28]:
metric_df.sort_values('acc-validate', ascending=False).head(15)

,model,accuracy,acc-validate,difference
50,decistion_tree-tfidf_1gram_4depth,0.899713,0.91,0.010287
52,decistion_tree-tfidf_1gram_6depth,0.939828,0.90,-0.039828
2,decistion_tree-cv_1gram_4depth,0.882521,0.90,0.017479
5,decistion_tree-cv_1gram_7depth,0.954155,0.90,-0.054155
53,decistion_tree-tfidf_1gram_7depth,0.957020,0.90,-0.057020
49,decistion_tree-tfidf_1gram_3depth,0.868195,0.90,0.031805
1,decistion_tree-cv_1gram_3depth,0.868195,0.90,0.031805
51,decistion_tree-tfidf_1gram_5depth,0.919771,0.88,-0.039771
3,decistion_tree-cv_1gram_5depth,0.914040,0.88,-0.034040
4,decistion_tree-cv_1gram_6depth,0.936963,0.88,-0.056963


### Ran 96 models

### decistion_tree-tfidf_1gram_4depth is our best model with 
- training-accuracy: 0.899713	
- validation-accuracy: 0.91	
- accuracy difference: 0.010287

- decistion_tree-tfidf_1gram_4depth	0.899713	0.91	0.010287
- decistion_tree-tfidf_1gram_6depth	0.939828	0.90	-0.039828
- decistion_tree-cv_1gram_4depth	0.882521	0.90	0.017479

In [73]:
def get_model_tree_tfidf_1gram_4depth(X_train, X_validate, y_train, y_validate):
    '''
    This will display metrics for a Descistion Tree classifier model using TfidfVectorizer
    with unigrams, and a max depth of 4.
    '''
    # create the TfidfVectorizer object with 1grams
    tfidf = TfidfVectorizer(ngram_range=(1, 1))
    # fit the tfid object, and transform the x_train dataset
    X_bow = tfidf.fit_transform(X_train)
    # transfrom the x_validate dataset
    X_val_bow = tfidf.transform(X_validate)
    # create a decision tree model with max depth 4
    tree = DecisionTreeClassifier(max_depth=4)
    # fit the decision tree on the training data
    tree.fit(X_bow, y_train)
    # get the accuracy score for the training data
    accuracy = tree.score(X_bow, y_train)
    # get the accuracy score for the validation data
    acc_validate = tree.score(X_val_bow, y_validate)
    # display the model metrics
    # display the model name and perameters
    print('Model : Decistion_Tree : TfidfVectorizer : 1gram : 4_max_depth')
    # display the accuracy on the training data
    print(f'Train accuracy : {accuracy}'),
    # display the accuracy on the validation data
    print(f'Validation accuracy : {acc_validate}'),
    # display the accuracy difference
    print(f'Difference : {acc_validate - accuracy}')
    # return the model for potential use on test data
    return tree, tfidf

In [74]:
def get_model_tree_tfidf_1gram_6depth(X_train, X_validate, y_train, y_validate):
    '''
    This will display metrics for a Descistion Tree classifier model using TfidfVectorizer
    with unigrams, and a max depth of 6.
    '''
    # create the TfidfVectorizer object with 1grams
    tfidf = TfidfVectorizer(ngram_range=(1, 1))
    # fit the tfid object, and transform the x_train dataset
    X_bow = tfidf.fit_transform(X_train)
    # transfrom the x_validate dataset
    X_val_bow = tfidf.transform(X_validate)
    # create a decision tree model with max depth 6
    tree = DecisionTreeClassifier(max_depth=6)
    # fit the decision tree on the training data
    tree.fit(X_bow, y_train)
    # get the accuracy score for the training data
    accuracy = tree.score(X_bow, y_train)
    # get the accuracy score for the validation data
    acc_validate = tree.score(X_val_bow, y_validate)
    # display the model metrics
    # display the model name and perameters
    print('Model : Decistion_Tree : TfidfVectorizer : 1gram : 6_max_depth')
    # display the accuracy on the training data
    print(f'Train accuracy : {accuracy}'),
    # display the accuracy on the validation data
    print(f'Validation accuracy : {acc_validate}'),
    # display the accuracy difference
    print(f'Difference : {acc_validate - accuracy}')
    # return the model for potential use on test data
    return tree, tfidf

In [75]:
def get_model__tree_cv_1gram_4depth(X_train, X_validate, y_train, y_validate):
    '''
    This will display metrics for a Descistion Tree classifier model using CountVectorizer
    with unigrams, and a max depth of 4.
    '''
    # create the CountVectorizer object with 1grams
    cv = CountVectorizer(ngram_range=(1, 1))
    # fit the tfid object, and transform the x_train dataset
    X_bow = cv.fit_transform(X_train)
    # transfrom the x_validate dataset
    X_val_bow = cv.transform(X_validate)
    # create a decision tree model with max depth 4
    tree = DecisionTreeClassifier(max_depth=4)
    # fit the decision tree on the training data
    tree.fit(X_bow, y_train)
    # get the accuracy score for the training data
    accuracy = tree.score(X_bow, y_train)
    # get the accuracy score for the validation data
    acc_validate = tree.score(X_val_bow, y_validate)
    # display the model metrics
    # display the model name and perameters
    print('Model : Decistion_Tree : CountVectorizer : 1gram : 4_max_depth')
    # display the accuracy on the training data
    print(f'train accuracy : {accuracy}'),
    # display the accuracy on the validation data
    print(f'validation accuracy : {acc_validate}'),
    # display the accuracy difference
    print(f'difference : {acc_validate - accuracy}')
    # return the model for potential use on test data
    return tree, cv

In [76]:
dt_t_1_4, tfidf_1 = get_model_tree_tfidf_1gram_4depth(
    X_train, X_validate, y_train, y_validate)

Model : Decistion_Tree : TfidfVectorizer : 1gram : 4_max_depth
Train accuracy : 0.8997134670487106
Validation accuracy : 0.89
Difference : -0.009713467048710567


In [77]:
dt_t_1_6, tfidf_1 = get_model_tree_tfidf_1gram_6depth(
    X_train, X_validate, y_train, y_validate)

Model : Decistion_Tree : TfidfVectorizer : 1gram : 6_max_depth
Train accuracy : 0.9398280802292264
Validation accuracy : 0.91
Difference : -0.02982808022922634


In [78]:
dt_c_1_4, cv_1 = get_model__tree_cv_1gram_4depth(
    X_train, X_validate, y_train, y_validate)

Model : Decistion_Tree : CountVectorizer : 1gram : 4_max_depth
train accuracy : 0.8825214899713467
validation accuracy : 0.89
difference : 0.007478510028653296


## Test dataset

In [79]:
def get_model_test_tfifd_1(X_train, y_train, X_test, y_test, model, vectorizer):
    '''
    This will use the passed classification model and vectorizer on the test dataset
    '''
    # use the vectorizer to transfor the test data
    X_test_bow = vectorizer.transform(X_test)
    # use the model to get an accuracy score on the test data
    acc_test = model.score(X_test_bow, y_test)
    # display the model metrics
    # display the model info
    print('Test Dataset')
    print('Model : Decistion_Tree : TfidfVectorizer : 1gram : 4_max_depth')
    # display the model accuracy score
    print(f'Test accuracy : {acc_test}')

In [80]:
get_model_test_tfifd_1(X_train, y_train, X_test, y_test, dt_t_1_4, tfidf_1)

Test Dataset
Model : Decistion_Tree : TfidfVectorizer : 1gram : 4_max_depth
Test accuracy : 0.78
